In [1]:
import pandas as pd
import sys
sys.path.append("../utils")

from utils import load_csv, save_data

In [7]:
df = load_csv("../../data/raw/fpt.csv")
df.head()

2025-05-26 10:39:12,128 - INFO - ✅ Load CSV thành công: ../../data/raw/fpt.csv | shape = (4593, 6)


,date,open,high,low,close,volume
0,NaN,NaN,NaN,NaN,NaN,NaN
1,23/05/2025,"116,700","117,800","116,000","117,100","4,440,500"
2,22/05/2025,"117,100","117,600","115,600","116,800","6,350,700"
3,21/05/2025,"120,000","120,000","117,000","118,300","6,024,400"
4,20/05/2025,"116,800","119,300","116,800","119,200","7,911,300"


In [8]:
df.dtypes

date      object
open      object
high      object
low       object
close     object
volume    object
dtype: object

In [9]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    original_rows = len(df)

    # Xoá các dòng toàn bộ là NaN hoặc chuỗi rỗng
    df = df.dropna(how='all')
    df = df[~(df.apply(lambda row: row.astype(str).str.strip().eq('').all(), axis=1))]
    
    # Chuẩn hóa tên cột
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

    # Xoá trùng lặp
    df = df.drop_duplicates()

    # Chuẩn hóa kiểu dữ liệu
    df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y")

    for col in ['open', 'high', 'low', 'close']:
        df[col] = df[col].str.replace(",", "").astype(float)
        df[col] = (df[col] / 1000).round(2)

    df['volume'] = df['volume'].str.replace(",", "").astype(int)


    # Điền missing
    for col in df.columns:
        if df[col].dtype in [int, float, float]:
            df[col] = df[col].fillna(df[col].mean())
        else:
            df[col] = df[col].fillna(df[col].mode().iloc[0])

    return df


In [10]:
# Tạo danh sách mã cổ phiếu cần xử lý
symbols = ["fpt", "hpg", "vnm"]
dfs = {}

# Lặp qua từng mã, load và tiền xử lý
for symbol in symbols:
    raw_path = f"../../data/raw/{symbol}.csv"
    df = load_csv(raw_path)
    df_clean = preprocess(df)
    dfs[symbol] = df_clean
    print(f"✅ {symbol.upper()} - Rows: {df_clean.shape[0]}, Columns: {df_clean.shape[1]}")


2025-05-26 10:39:19,202 - INFO - ✅ Load CSV thành công: ../../data/raw/fpt.csv | shape = (4593, 6)


2025-05-26 10:39:20,487 - INFO - ✅ Load CSV thành công: ../../data/raw/hpg.csv | shape = (4364, 6)


✅ FPT - Rows: 4592, Columns: 6


2025-05-26 10:39:21,525 - INFO - ✅ Load CSV thành công: ../../data/raw/vnm.csv | shape = (4819, 6)


✅ HPG - Rows: 4363, Columns: 6
✅ VNM - Rows: 4818, Columns: 6


In [11]:
# Hiển thị 5 dòng đầu tiên của mỗi mã cổ phiếu
for symbol in symbols:
    print(f"\n{symbol.upper()} - Dữ liệu sau xử lý:")
    display(dfs[symbol].head())



FPT - Dữ liệu sau xử lý:


,date,open,high,low,close,volume
1,2025-05-23,116.7,117.8,116.0,117.1,4440500
2,2025-05-22,117.1,117.6,115.6,116.8,6350700
3,2025-05-21,120.0,120.0,117.0,118.3,6024400
4,2025-05-20,116.8,119.3,116.8,119.2,7911300
5,2025-05-19,118.8,120.5,116.6,118.2,9916700



HPG - Dữ liệu sau xử lý:


,date,open,high,low,close,volume
1,2025-05-23,25.75,25.85,25.55,25.60,11096000
2,2025-05-22,25.75,25.95,25.55,25.65,24363700
3,2025-05-21,25.75,26.10,25.65,25.75,20150500
4,2025-05-20,25.60,25.80,25.60,25.65,17449000
5,2025-05-19,25.75,25.95,25.55,25.65,14995700



VNM - Dữ liệu sau xử lý:


,date,open,high,low,close,volume
1,2025-05-23,55.2,55.5,55.1,55.3,1666700
2,2025-05-22,55.4,55.6,55.0,55.0,2665200
3,2025-05-21,56.0,56.0,55.4,55.4,4464400
4,2025-05-20,56.1,56.3,55.8,55.9,3224100
5,2025-05-19,56.2,57.0,56.0,56.0,2795600


In [12]:
# Save
for symbol in symbols:
    path = f"../../data/clean/{symbol}.csv"
    save_data(dfs[symbol], f"../../data/clean/{symbol}.csv")

2025-05-26 10:39:37,305 - INFO - 💾 Đã lưu dữ liệu thành công: ../../data/clean/fpt.csv
2025-05-26 10:39:37,375 - INFO - 💾 Đã lưu dữ liệu thành công: ../../data/clean/hpg.csv
2025-05-26 10:39:37,411 - INFO - 💾 Đã lưu dữ liệu thành công: ../../data/clean/vnm.csv


In [ ]:
%pip install pymongo
from pymongo import MongoClient, errors

In [14]:
# Kết nối MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client['vn_project']

for symbol in symbols:
    collection = db[symbol]

    # Đảm bảo 'date' là unique (chỉnh lại nếu bạn dùng cột khác làm unique)
    collection.create_index('date', unique=True)

    records = dfs[symbol].to_dict(orient='records')
    
    success = 0
    for record in records:
        try:
            collection.insert_one(record)
            success += 1
        except errors.DuplicateKeyError:
            continue  # Bỏ qua bản ghi đã tồn tại

    print(f"Symbol {symbol.upper()}: đã thêm {success} bản ghi mới.")


Symbol FPT: đã thêm 4592 bản ghi mới.
Symbol HPG: đã thêm 4363 bản ghi mới.
Symbol VNM: đã thêm 4818 bản ghi mới.
